In [ ]:
!pip install python-dotenv \
    langchain \
    langchain-community \
    langchain-core \
    langchain-openai \
    chromadb \
    openai

In [ ]:
# 导入 os 模块，用于处理文件路径和目录
import os

# 从 langchain 库的 hub 模块导入 hub，用于访问 prompt 的 hub
from langchain import hub
# 从 langchain 库的 agents 模块导入 AgentExecutor 和 create_react_agent，用于创建和运行 ReAct 代理
from langchain.agents import AgentExecutor, create_react_agent
# 从 langchain 库的 chains 模块导入 create_history_aware_retriever 和 create_retrieval_chain，用于创建检索链
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
# 从 langchain 库的 chains.combine_documents 模块导入 create_stuff_documents_chain，用于创建 stuff 文档链
from langchain.chains.combine_documents import create_stuff_documents_chain
# 从 langchain_community 库的 vectorstores 模块导入 Chroma，用于使用 Chroma 向量数据库
from langchain_community.vectorstores import Chroma
# 从 langchain_core 库的 messages 模块导入 AIMessage 和 HumanMessage，用于表示 AI 消息和人类消息
from langchain_core.messages import AIMessage, HumanMessage
# 从 langchain_core 库的 prompts 模块导入 ChatPromptTemplate 和 MessagesPlaceholder，用于创建聊天提示模板和消息占位符
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# 从 langchain_core 库的 tools 模块导入 Tool，用于创建工具
from langchain_core.tools import Tool
# 从 langchain_openai 库导入 ChatOpenAI 和 OpenAIEmbeddings，用于使用 OpenAI 的聊天模型和嵌入模型
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
#

"""
1. 配置 Chroma 向量数据库目录

定义 Chroma 向量数据库的持久化存储目录。
向量数据库用于存储和检索文档嵌入，以便进行语义搜索和 RAG (Retrieval-Augmented Generation)。
"""
# Load the existing Chroma vector store
# 获取当前脚本的目录
current_dir = os.path.dirname(os.path.abspath(__file__))
# 构建数据库目录的路径，假设数据库位于父目录的父目录的 rag 目录下的 db 目录中
db_dir = os.path.join(current_dir, "..", "..", "rag", "db")
# 构建 Chroma 数据库持久化存储的完整路径
persistent_directory = os.path.join(db_dir, "chroma_db_with_metadata")

"""
检查并加载现有的 Chroma 向量数据库

检查指定的持久化目录是否存在 Chroma 向量数据库。
如果存在，则加载现有的数据库；如果不存在，则抛出 FileNotFoundError 异常，提示用户检查路径。
这避免了每次运行代码时都重新创建向量数据库，提高了效率。
"""
# Check if the Chroma vector store already exists
if os.path.exists(persistent_directory):
    print("Loading existing vector store...")
    # 如果目录存在，则加载已有的 Chroma 向量数据库，embedding_function 初始设置为 None，稍后会更新
    db = Chroma(persist_directory=persistent_directory,
                embedding_function=None)
else:
    # 如果目录不存在，抛出异常，提示用户检查路径
    raise FileNotFoundError(
        f"The directory {persistent_directory} does not exist. Please check the path."
    )

"""
2. 定义嵌入模型

使用 OpenAIEmbeddings 定义文档嵌入模型。
嵌入模型用于将文本转换为向量，以便在向量空间中进行相似性搜索。
这里使用 'text-embedding-3-small' 模型，这是一个性价比高的嵌入模型。
"""
# Define the embedding model
# 创建 OpenAIEmbeddings 实例，使用 'text-embedding-3-small' 模型
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

"""
3. 重新加载 Chroma 向量数据库并配置嵌入函数

使用之前定义的嵌入模型重新加载 Chroma 向量数据库。
在第一次加载时，embedding_function 设置为 None，现在使用 OpenAIEmbeddings 对象进行更新。
确保向量数据库使用正确的嵌入函数进行操作。
"""
# Load the existing vector store with the embedding function
# 重新加载 Chroma 向量数据库，这次使用 OpenAIEmbeddings 作为嵌入函数
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

"""
4. 创建检索器 (Retriever)

从 Chroma 向量数据库创建检索器。
检索器用于根据用户查询在向量数据库中搜索相关文档。
`search_type="similarity"` 指定使用相似性搜索，`search_kwargs={"k": 3}` 指定返回最相似的 3 个文档。
"""
# Create a retriever for querying the vector store
# 从 Chroma 数据库创建检索器，用于执行相似性搜索
# `search_type` 指定搜索类型为 "similarity" (相似性搜索)
# `search_kwargs` 设置搜索参数，`k=3` 表示返回最相关的 3 个文档
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

"""
5. 创建 ChatOpenAI 模型

初始化 ChatOpenAI 模型，使用 'gpt-4o' 模型。
这是用于生成答案和处理自然语言对话的大型语言模型。
'gpt-4o' 是一个先进的多模态模型，具有强大的文本处理能力。
"""
# Create a ChatOpenAI model
# 初始化 ChatOpenAI 模型，使用 'gpt-4o' 模型
# llm = ChatOpenAI(model="gpt-4o")

# OpenAI API调用（代理方式）
llm = ChatOpenAI(
    api_key="XXX",
    base_url="https://vip.apiyi.com/v1",
    model="gpt-4o"
)


"""
6. 上下文情境化问题提示 (Contextualize Question Prompt)

定义一个系统提示，用于指导 AI 如何根据聊天历史记录和最新的用户问题，生成一个独立的、无需上下文也能理解的问题。
这个提示的目的是让 AI 能够处理在对话中可能出现的指代和省略，确保检索器总是能获得清晰明确的查询。
"""
# Contextualize question prompt
# 系统提示，用于指导 AI 根据聊天历史和用户问题，生成一个独立的、无需上下文也能理解的问题
# 目标是让 AI 能够处理对话中的上下文依赖，生成清晰的查询
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)

"""
7. 创建上下文情境化问题提示模板

使用 ChatPromptTemplate.from_messages 创建提示模板，该模板包含系统提示、聊天历史记录的占位符和用户输入的占位符。
MessagesPlaceholder("chat_history") 用于在运行时动态地插入聊天历史记录。
"""
# Create a prompt template for contextualizing questions
# 使用 ChatPromptTemplate 创建提示模板，包含系统提示、聊天历史记录占位符和用户输入占位符
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

"""
8. 创建历史感知检索器 (History-Aware Retriever)

使用 create_history_aware_retriever 函数，将 LLM、基础检索器和上下文情境化问题提示模板组合在一起，创建一个历史感知检索器。
这个检索器能够在检索文档之前，先利用 LLM 和聊天历史记录来提炼用户的问题，从而提高检索的准确性。
"""
# Create a history-aware retriever
# 使用 create_history_aware_retriever 创建历史感知检索器
# 结合 LLM, 基础检索器和上下文情境化问题提示模板
# 使得检索器能够根据聊天历史记录来优化检索query
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

"""
9. 答案问题提示 (Answer Question Prompt)

定义一个系统提示，用于指导 AI 如何使用检索到的上下文来回答用户问题。
该提示指示 AI 作为一个问题解答助手，使用检索到的上下文，并在不知道答案时明确表示“我不知道”。
同时，限制答案长度为最多三句话，保持简洁。
"""
# Answer question prompt
# 系统提示，用于指导 AI 如何使用检索到的上下文来回答问题
# 指示 AI 作为问题解答助手，使用上下文，并在不知道答案时回答 "I don't know"
# 限制答案长度为最多三句话，保持简洁
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know. Use three sentences maximum and keep the answer "
    "concise."
    "\n\n"
    "{context}"
)

"""
10. 创建答案问题提示模板

使用 ChatPromptTemplate.from_messages 创建答案问题提示模板，包含系统提示、聊天历史记录占位符和用户输入占位符。
虽然答案问题提示模板中也包含了 `MessagesPlaceholder("chat_history")`，但在当前的 `create_stuff_documents_chain` 使用方式下，聊天历史记录实际上并没有直接被答案生成链使用。
这里保留 `MessagesPlaceholder("chat_history")` 可能是为了未来扩展，或者保持提示模板结构的一致性。
"""
# Create a prompt template for answering questions
# 使用 ChatPromptTemplate 创建答案问题提示模板，包含系统提示、聊天历史记录占位符和用户输入占位符
# 注意：虽然这里也包含了 chat_history 占位符，但在当前的 chain 结构中，答案生成链可能并没有直接使用 chat_history
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

"""
11. 创建文档组合链 (Combine Documents Chain)

使用 create_stuff_documents_chain 函数，将 LLM 和答案问题提示模板组合在一起，创建一个文档组合链。
`create_stuff_documents_chain` 使用 "stuff" 方式，将所有检索到的文档合并成一个字符串，一次性送入 LLM 进行处理。
这个链负责接收检索器返回的文档，并将它们与用户问题一起传递给 LLM，以生成最终答案。
"""
# Create a chain to combine documents for question answering
# 使用 create_stuff_documents_chain 创建文档组合链
# `create_stuff_documents_chain` 使用 "stuff" 方式，将所有检索到的文档一次性喂给 LLM
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

"""
12. 创建 RAG 链 (Retrieval-Augmented Generation Chain)

使用 create_retrieval_chain 函数，将历史感知检索器和文档组合链组合在一起，创建 RAG 链。
RAG 链是整个问答系统的核心，它首先使用历史感知检索器检索相关文档，然后使用文档组合链和 LLM 根据检索到的文档生成答案。
"""
# Create a retrieval chain that combines the history-aware retriever and the question answering chain
# 使用 create_retrieval_chain 创建 RAG 链，将历史感知检索器和答案生成链组合起来
rag_chain = create_retrieval_chain(
    history_aware_retriever, question_answer_chain)


"""
13. 设置带有文档存储检索器的 ReAct Agent (ReAct Agent with Document Store Retriever)

配置 ReAct Agent，使其能够利用文档存储进行检索和问题回答。
ReAct (Reason and Act) 是一种 agent 框架，它允许 agent 在思考 (Reason) 和执行动作 (Act) 之间交替进行，从而更有效地完成复杂任务。
这里使用预定义的 ReAct 提示 (react_docstore_prompt) 和一个工具 (Answer Question Tool)。
"""
# Set Up ReAct Agent with Document Store Retriever
# 加载 ReAct Docstore Prompt
react_docstore_prompt = hub.pull("hwchase17/react")

"""
14. 定义工具 (Tools)

创建一个工具列表，ReAct Agent 可以使用这些工具来执行不同的操作。
这里定义了一个名为 "Answer Question" 的工具，该工具使用之前创建的 RAG 链 (rag_chain) 来回答问题。
工具的描述 (description) 非常重要，ReAct Agent 会根据描述来决定何时以及如何使用工具。
"""
# 定义 Agent 可以使用的工具列表
tools = [
    Tool(
        name="Answer Question", # 工具名称，Agent 会根据名称来调用工具
        # 工具的函数，这里使用 rag_chain.invoke 来执行 RAG 链，回答问题
        func=lambda input, **kwargs: rag_chain.invoke(
            {"input": input, "chat_history": kwargs.get("chat_history", [])}
        ),
        # 工具的描述，描述工具的用途，ReAct Agent 会根据描述来决定何时使用该工具
        description="useful for when you need to answer questions about the context",
    )
]

"""
15. 创建 ReAct Agent

使用 create_react_agent 函数创建 ReAct Agent。
需要传入 LLM, 工具列表和 ReAct 提示模板。
ReAct Agent 会根据提示模板和可用的工具，以及用户的输入，自主决定执行哪些操作来完成任务。
"""
# Create the ReAct Agent with document store retriever
# 使用 create_react_agent 创建 ReAct Agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=react_docstore_prompt,
)

"""
16. 创建 Agent 执行器 (Agent Executor)

使用 AgentExecutor.from_agent_and_tools 创建 Agent 执行器。
Agent 执行器是运行 Agent 的核心组件，它负责接收用户输入，调用 Agent 进行决策，执行工具，并返回最终结果。
`handle_parsing_errors=True` 用于处理 Agent 输出解析错误的情况，`verbose=True` 开启详细日志输出，方便调试。
"""
# 创建 AgentExecutor，用于运行 Agent
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, handle_parsing_errors=True, verbose=True,
)

"""
17. 主聊天循环 (Main Chat Loop)

实现一个无限循环，模拟聊天对话。
用户可以输入查询，Agent 执行器会处理查询并返回 AI 的回复。
聊天历史记录 (chat_history) 会在每次对话后更新，以便 Agent 可以感知上下文。
输入 "exit" 可以结束循环。
"""
# 初始化聊天历史记录列表
chat_history = []
# 进入无限循环，模拟聊天对话
while True:
    # 获取用户输入
    query = input("You: ")
    # 如果用户输入 "exit"，则退出循环
    if query.lower() == "exit":
        break
    # 调用 agent_executor 的 invoke 方法，传入用户输入和聊天历史记录，获取 AI 的回复
    response = agent_executor.invoke(
        {"input": query, "chat_history": chat_history})
    # 打印 AI 的回复
    print(f"AI: {response['output']}")

    # 更新聊天历史记录，将用户消息和 AI 消息添加到列表中
    chat_history.append(HumanMessage(content=query))
    chat_history.append(AIMessage(content=response["output"]))